In [2]:
import tensorflow as tf

ftGoals = tf.constant([[2,3], [3,4], [2,2]])

ftGoals = tf.cast(ftGoals, dtype=tf.int32)  # (batch, 2)
h = (tf.math.greater(ftGoals[..., 0], ftGoals[..., 1]), tf.math.equal(ftGoals[..., 0], ftGoals[..., 1]), tf.math.less(ftGoals[..., 0], ftGoals[..., 1]))
h = tf.cast(tf.transpose(h), dtype=tf.float32)  # (batch, nQueries)

print(h)

tf.Tensor(
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]], shape=(3, 3), dtype=float32)


In [4]:
import pandas as pd
d = { 'c1': [1,2,3], 'c2': [4,5,6], 'c3': [7,8,9]}
df = pd.DataFrame(d)
print(df)
try: df.drop('c4', axis=1)
except: pass
print(df)

   c1  c2  c3
0   1   4   7
1   2   5   8
2   3   6   9
   c1  c2  c3
0   1   4   7
1   2   5   8
2   3   6   9


In [8]:
import tensorflow as tf
import numpy as np

def bell(x, mu, sigma):
    x = tf.norm(x)
    return tf.math.exp(- tf.math.pow((x-mu)/sigma, 2) / 2) / (sigma * tf.math.sqrt(np.pi*2))

print(bell(tf.constant([1.0, 1.0]), 1, 1))

tf.Tensor(0.36614534, shape=(), dtype=float32)


In [ ]:
import datetime
config = {}
config['baseDate'] = datetime.datetime(2000, 1, 1)
print(datetime.datetime.now().year - config['baseDate'].year)

In [ ]:
import tensorflow as tf
pcode = tf.constant([[1,2,3], [4,5,6], [7,8,9]], dtype=tf.int32)
pos = tf.constant([0,2], dtype=tf.int32)

print(tf.gather(pcode, pos))

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def find_extrapolation(sequence, tail_data_len):
    assert tail_data_len <= len(sequence)
    x = np.array([[i] for i in range(tail_data_len)], dtype=np.float32)
    y = np.array(sequence[- tail_data_len :], dtype=np.float32)
    model = LinearRegression()
    model.fit(x, y)
    extrapolation = model.predict(np.array([[tail_data_len]], dtype=np.float32))[0]
    return extrapolation

intervals_a = [1,2,3,4.3]
e = find_extrapolation(intervals_a, 3)
print(e)


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from config import config
import data_helpers
import datetime

In [ ]:
start = datetime.datetime.now()
print(start)

In [ ]:
df = data_helpers.read_large_excel(config['data'], "contest_total_data")

In [ ]:
df = data_helpers.improve_contest_data(df)

In [ ]:
gameGraph = data_helpers.createGameGraph(df, renew=True)
(a, b) = list(gameGraph.edges())[0]
print(gameGraph[a][b]['games'])
L = [(ts, id) for (id, ts) in gameGraph[a][b]['games']]
print(L)
L.sort(reverse=True)
print(L)
L = [(id, ts) for (ts, id) in L]
print(L)


In [ ]:
print(len(list(gameGraph.nodes)), len(list(gameGraph.edges)))

In [ ]:
# Although know max conductivity of a single game is 1.0, we start from b_conductance_search of more than 1.0
# bacause multiple games between a home and a away might increase conductance between them to more than 1.0.

bookie_profit_percent=5.0
a_conductance_search=0.01
b_conductance_search=15.0
conductance365=0.9      # the larger, the fresher it remember the past games
conductanceMedTeam=0.8  # the larger, the wider spanning tree. very sensitive for eta.
ids_max = 10           #
minConductanceStep = 0.000005         
renew=True

fixture_id_to_ids = data_helpers.fixture_id_to_ids(
        df,
        gameGraph,
        a_conductance_search = a_conductance_search,
        b_conductance_search = b_conductance_search,
        conductance365 = conductance365, 
        conductanceMedTeam = conductanceMedTeam,
        ids_max = ids_max,
        minConductanceStep = minConductanceStep,
        renew = renew,
        )
end = datetime.datetime.now()
hours = round((end-start).total_seconds()/60/60, 2)
nDataPoints = len(list(fixture_id_to_ids.keys()))
maxLen = max([len(ids) for ids in fixture_id_to_ids.values()])
filename = data_helpers.dataPrams_to_filename(
        bookie_profit_percent,
        a_conductance_search, b_conductance_search, conductance365, 
        conductanceMedTeam, ids_max, minConductanceStep, hours, nDataPoints, maxLen)
path = os.path.join('./data', filename + '.json')
data_helpers.SaveJsonData(fixture_id_to_ids, path)

# filename = 'ids-0.01-15.0-0.9-0.8-130-0.000005-22.28-32601-140'
(bookie_profit_percent, a_conductance_search, b_conductance_search, conductance365, 
        conductanceMedTeam, ids_max, minConductanceStep, hours, nDataPoints, maxLen) \
= data_helpers.filename_to_dataParams(filename)
path = os.path.join('./data', filename + '.json')
fixture_id_to_ids = data_helpers.LoadJsonData(path)


In [ ]:
lengths = [len(ids) for ids in fixture_id_to_ids.values()]
plt.hist(lengths, np.linspace(0, 200, 101))
plt.ylim(plt.ylim())
maxLen = max(lengths)
plt.plot([maxLen, maxLen], plt.ylim())
plt.title(f'Max length of ids: {maxLen}')

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def getPair(df, baseId):
    frame = df.loc[df['fixture_id'] == baseId, ['teams_home_team_id', 'teams_away_team_id']]
    pairs = list(zip(list(frame['teams_home_team_id']), list(frame['teams_away_team_id'])))
    return pairs[0]

def showGraph(n):
    if(len(list(fixture_id_to_ids.items())) > n):
        (baseId, ids) = list(fixture_id_to_ids.items())[n]
        print('base pair: ', baseId, getPair(df, int(baseId)), ids)     # No int(.), no work.
        
        if baseId in ids:
            raise("BaseId in ids")

        edges = []
        for id in ids:
            edges.append(getPair(df, id))
        print('baseId, nGames, pairs: ', baseId, len(ids), edges)

        G = nx.Graph()
        G.add_edges_from(edges)

        subax1 = plt.subplot(111)
        nx.draw(G, with_labels=True)
        # print('G.nodes: ', G.nodes)
        print('G.edges: ', G.edges)

In [ ]:
idx = 25002

In [ ]:
# length = len(list(fixture_id_to_ids.keys()))
# for idx in range(length):
showGraph(idx)
idx += 10
print(idx)